## Unsupervised Learning Trading Strategy
- Download/Load SP500 stocks prices data.
- Calculate different features and indicators on each stock.
- Aggregate on monthly level and filter top 150 most liquid stocks.
- Calculate Monthly Returns for different time-horizons.
- Download Fama-French Factors and Calculate Rolling Factor Betas.
- For each month fit a K-Means Clustering Algorithm to group similar assets based on their features.
- For each month select assets based on the cluster and form a portfolio based on Efficient Frontier max sharpe ratio optimization.
- Visualize Portfolio returns and compare to SP500 returns.

## All Packages Needed:
- pandas, numpy, matplotlib, statsmodels, pandas_datareader, datetime, yfinance, sklearn, PyPortfolioOpt

In [1]:
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install statsmodels
%pip install pandas-datareader
%pip install datetime
%pip install yfinance
%pip install scikit-learn
%pip install PyPortfolioOpt
%pip install pandas_ta

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.2 MB/s eta 0:00:000:00:01m eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=378c55cbd1da3eb0f879cadcaee8c1c72fcc31cdc160f311ff8748afe113d943
  Stored i

1. Download/Load SP500 stocks prices data:

In [9]:
# from statsmodels.regression.rolling import RollingOLS
# import pandas_datareader.data as web
# import matplotlib.pyplot as plt
# import statsmodels.api as sm
# import pandas as pd
# import numpy as np
# import datetime as dt
# import yfinance as yf
# import pandas_ta
# import warnings
# warnings.filterwarnings('ignore')

# sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S&P_500_companies')[0]

# sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

# symbols_list = sp500['Symbol'].unique().tolist()

# end_date = '2023-11-21'

# start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

# df = yf.download(tickers=symbols_list, start=start_date, end=end_date).stack()

# df.index.names = ['date', 'ticker']

# df.columns = df.columns.str.lower()

df

adj close       close        high         low        open  \
date       ticker                                                               
2015-11-23 A        37.476860   40.029999   40.070000   39.139999   39.290001   
           AAL      40.423809   42.299999   42.740002   41.810001   42.110001   
           AAPL     26.799698   29.437500   29.932501   29.334999   29.817499   
           ABBV     43.283791   61.150002   61.529999   60.840000   61.310001   
           ABT      39.117046   45.439999   46.099998   45.259998   46.029999   
...                       ...         ...         ...         ...         ...   
2023-11-20 YUM     128.119995  128.119995  128.580002  127.300003  127.940002   
           ZBH     112.500000  112.500000  112.650002  111.339996  111.620003   
           ZBRA    220.539993  220.539993  221.779999  216.600006  218.839996   
           ZION     35.779999   35.779999   36.169998   35.560001   36.169998   
           ZTS     176.059998  176.059998  177.000000  173.899994  174.320007   

                        volume  
date       ticker               
2015-11-23 A         6473400.0  
           AAL       4166100.0  
           AAPL    129930000.0  
           ABBV      6405000.0  
           ABT       4593800.0  
...                        ...  
2023-11-20 YUM        915000.0  
           ZBH       1345100.0  
           ZBRA       330600.0  
           ZION      1589000.0  
           ZTS       1056700.0  

[994838 rows x 6 columns]

2. Calculate features and technical indicators for each stock.
   - Garman-Klass Volatility
   - RSI
   - Bollinger Bands
   - ATR
   - MACD
   - Dollar Volume

In [23]:
# df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2 - (2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))*2)

# df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

# df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])

# df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])

# df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

# def compute_atr(stock_data):
#   atr = pandas_ta.atr(high=stock_data['high'],
#                       low=stock_data['low'],
#                       close=stock_data['close'],
#                       length=14)
#   return atr.sub(atr.mean()).div(atr.std())

# df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

# def compute_macd(close):
#   macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
#   return macd.sub(macd.mean()).div(macd.std())

# df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

# df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

adj close       close        high         low        open  \
date       ticker                                                               
2015-11-23 A        37.476860   40.029999   40.070000   39.139999   39.290001   
           AAL      40.423809   42.299999   42.740002   41.810001   42.110001   
           AAPL     26.799698   29.437500   29.932501   29.334999   29.817499   
           ABBV     43.283791   61.150002   61.529999   60.840000   61.310001   
           ABT      39.117046   45.439999   46.099998   45.259998   46.029999   
...                       ...         ...         ...         ...         ...   
2023-11-20 YUM     128.119995  128.119995  128.580002  127.300003  127.940002   
           ZBH     112.500000  112.500000  112.650002  111.339996  111.620003   
           ZBRA    220.539993  220.539993  221.779999  216.600006  218.839996   
           ZION     35.779999   35.779999   36.169998   35.560001   36.169998   
           ZTS     176.059998  176.059998  177.000000  173.899994  174.320007   

                        volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-11-23 A         6473400.0          0.036778        NaN       NaN   
           AAL       4166100.0          0.031815        NaN       NaN   
           AAPL    129930000.0          0.082642        NaN       NaN   
           ABBV      6405000.0          0.269052        NaN       NaN   
           ABT       4593800.0          0.125896        NaN       NaN   
...                        ...               ...        ...       ...   
2023-11-20 YUM        915000.0         -0.001036  59.441962  4.778174   
           ZBH       1345100.0         -0.005999  56.139679  4.623112   
           ZBRA       330600.0         -0.005699  51.588388  5.286738   
           ZION      1589000.0          0.008520  57.703637  3.372132   
           ZTS       1056700.0         -0.007517  56.890496  5.026941   

                     bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2015-11-23 A            NaN       NaN       NaN       NaN     242.602706  
           AAL          NaN       NaN       NaN       NaN     168.409631  
           AAPL         NaN       NaN       NaN       NaN    3482.084745  
           ABBV         NaN       NaN       NaN       NaN     277.232679  
           ABT          NaN       NaN       NaN       NaN     179.695888  
...                     ...       ...       ...       ...            ...  
2023-11-20 YUM     4.828897  4.879620  0.199591  0.895190     117.229796  
           ZBH     4.680042  4.736972 -0.254995  0.202355     151.323750  
           ZBRA    5.345200  5.403662 -0.012923 -0.160356      72.910522  
           ZION    3.515364  3.658596 -0.092737  0.696147      56.854418  
           ZTS     5.115873  5.204804  0.963583  0.488186     186.042599  

[994838 rows x 14 columns]

3. Aggregate to monthly level and filter top 150 most liquid stocks for each month.
   - To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency.

In [33]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open', 'high', 'low', 'close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
           df.unstack()[last_cols].resample('M').last().stack('ticker')],
           axis=1)).dropna()

data

dollar_volume   adj close  garman_klass_vol        rsi  \
date       ticker                                                           
2015-12-31 A          103.333138   39.250355          0.050582  57.183815   
           AAL        319.493870   40.471588          0.038663  47.951480   
           AAPL      4258.527066   23.956985          0.085487  30.946429   
           ABBV       294.510530   41.931839          0.270428  52.892452   
           ABT        211.476879   38.660789          0.119282  48.262388   
...                          ...         ...               ...        ...   
2023-11-30 YUM        181.963715  128.119995         -0.001036  59.441962   
           ZBH        239.748735  112.500000         -0.005999  56.139679   
           ZBRA       106.252939  220.539993         -0.005699  51.588388   
           ZION        71.560611   35.779999          0.008520  57.703637   
           ZTS        438.153718  176.059998         -0.007517  56.890496   

                     bb_low    bb_mid   bb_high       atr      macd  
date       ticker                                                    
2015-12-31 A       3.644514  3.676936  3.709358 -1.161306  0.095303  
           AAL     3.695286  3.743884  3.792481  0.607890  0.112605  
           AAPL    3.194346  3.269361  3.344377 -1.002304 -0.730643  
           ABBV    3.671835  3.723142  3.774448 -1.067619 -0.010320  
           ABT     3.656643  3.682609  3.708575 -1.061455 -0.189849  
...                     ...       ...       ...       ...       ...  
2023-11-30 YUM     4.778174  4.828897  4.879620  0.199591  0.895190  
           ZBH     4.623112  4.680042  4.736972 -0.254995  0.202355  
           ZBRA    5.286738  5.345200  5.403662 -0.012923 -0.160356  
           ZION    3.372132  3.515364  3.658596 -0.092737  0.696147  
           ZTS     5.026941  5.115873  5.204804  0.963583  0.488186  

[47458 rows x 9 columns]

  - Calculate 5-year rolling average of dollar volume for each stocks before filtering.

In [40]:
# data['dollar_volume'] = (data['dollar_volume'].unstack('ticker').rolling(5*12).mean().stack())

# data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

# data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2020-11-30 AAL      14.130000          0.044575  58.332321  2.440051   
           AAPL    116.983948          0.000520  54.008266  4.698839   
           ABBV     92.321564          0.100445  71.866658  4.373911   
           ABT     102.758202          0.036883  48.977968  4.621924   
           ACN     238.940918          0.032185  61.931851  5.376573   
...                       ...               ...        ...       ...   
2023-11-30 VRTX    356.839996         -0.015221  47.397467  5.843848   
           VZ       36.730000         -0.009356  67.482270  3.547485   
           WFC      42.790001          0.001316  59.733474  3.660538   
           WMT     155.300003          0.001214  38.783824  5.054606   
           XOM     104.500000          0.006574  45.335922  4.627209   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2020-11-30 AAL     2.613901  2.787751 -0.786781  0.861154  
           AAPL    4.751182  4.803526  0.673168 -0.077952  
           ABBV    4.471928  4.569946  0.049640  2.112393  
           ABT     4.663147  4.704369  1.021236 -0.256770  
           ACN     5.443973  5.511372  0.230954  1.248935  
...                     ...       ...       ...       ...  
2023-11-30 VRTX    5.905214  5.966581  1.628970 -0.687191  
           VZ      3.594962  3.642439 -0.832373  1.943153  
           WFC     3.729717  3.798896 -0.753369  0.773765  
           WMT     5.101774  5.148942  0.981298 -0.649294  
           XOM     4.660664  4.694120  0.645576 -1.352009  

[5513 rows x 8 columns]

4. Calculate Monthly Returns for different time horizons as features.
   - To capture time series dynamics that reflect, for example, momentum patterns, we compute historical returns using the method .pct_change(lag), that is, returns over various monthly periods as identified by lags.

In [45]:
def calculate_returns(df):

  outlier_cutoff = 0.005

  lags = [1, 2, 3, 6, 9, 12]

  for lag in lags:
    df[f'return_{lag}m'] = (df['adj close']
                          .pct_change(lag)
                          .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                  upper=x.quantile(1-outlier_cutoff)))
                          .add(1)
                          .pow(1/lag)
                          .sub(1))
  return df

data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2022-01-31 AAL      16.469999         -0.042152  42.119499  2.789219   
           AAPL    172.805557         -0.011411  56.861830  5.051317   
           ABBV    127.954788          0.051820  64.192417  4.819810   
           ABT     123.331398          0.013840  46.477958  4.751287   
           ACN     344.256073          0.002092  45.541295  5.733298   
...                       ...               ...        ...       ...   
2023-11-30 VRTX    356.839996         -0.015221  47.397467  5.843848   
           VZ       36.730000         -0.009356  67.482270  3.547485   
           WFC      42.790001          0.001316  59.733474  3.660538   
           WMT     155.300003          0.001214  38.783824  5.054606   
           XOM     104.500000          0.006574  45.335922  4.627209   

                     bb_mid   bb_high       atr      macd  return_1m  \
date       ticker                                                      
2022-01-31 AAL     2.925444  3.061668 -0.415825 -0.635583  -0.082962   
           AAPL    5.128811  5.206305  1.973336 -1.031766  -0.015712   
           ABBV    4.842912  4.866015  0.591804  0.914834   0.021496   
           ABT     4.835107  4.918927  1.710540 -2.432121  -0.089301   
           ACN     5.859449  5.985600  2.992092 -3.344795  -0.139653   
...                     ...       ...       ...       ...        ...   
2023-11-30 VRTX    5.905214  5.966581  1.628970 -0.687191  -0.014554   
           VZ      3.594962  3.642439 -0.832373  1.943153   0.045545   
           WFC     3.729717  3.798896 -0.753369  0.773765   0.085529   
           WMT     5.101774  5.148942  0.981298 -0.649294  -0.049630   
           XOM     4.660664  4.694120  0.645576 -1.352009  -0.003726   

                   return_2m  return_3m  return_6m  return_9m  return_12m  
date       ticker                                                          
2022-01-31 AAL     -0.035099  -0.049838  -0.034879  -0.030276   -0.003463  
           AAPL     0.028275   0.053265   0.031114   0.032687    0.024220  
           ABBV     0.095344   0.064474   0.031379   0.026867    0.028315  
           ABT      0.008470  -0.002542   0.009975   0.007900    0.003776  
           ACN     -0.004070  -0.004005   0.018947   0.023233    0.031799  
...                      ...        ...        ...        ...         ...  
2023-11-30 VRTX     0.013000   0.008069   0.016446   0.023199    0.010074  
           VZ       0.071290   0.023586   0.011612   0.000022    0.000952  
           WFC      0.027896   0.014964   0.014919  -0.007139   -0.006896  
           WMT     -0.014581  -0.015215   0.009942   0.011178    0.002830  
           XOM     -0.052961  -0.017495   0.006642  -0.002726   -0.002330  

[3273 rows x 14 columns]